In [1]:
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

import evaluate
import numpy as np
import torch
import torch.nn as nn
from datasets import load_dataset
from peft import LoraConfig, TaskType, get_peft_model
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    HfArgumentParser,
    PreTrainedTokenizerBase,
    Trainer,
    TrainingArguments,
)
from transformers.utils import PaddingStrategy



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/mymusise/pro/stable-diffusion-webui/venv/lib/python3.8/site-packages/bitsandbytes-0.39.0-py3.8.egg/bitsandbytes/libbitsandbytes_cuda117.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/mymusise/pro/stable-diffusion-webui/venv/lib/python3.8/site-packages/bitsandbytes-0.39.0-py3.8.egg/bitsandbytes/libbitsandbytes_cuda117.so...


/home/mymusise/pro/stable-diffusion-webui/venv/lib/python3.8/site-packages/bitsandbytes-0.39.0-py3.8.egg/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/home/mymusise/pro/stable-diffusion-webui/venv/lib/python3.8/site-packages/bitsandbytes-0.39.0-py3.8.egg/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('"zh-cn","availableLanguages"'), PosixPath('"zh-cn"},"_languagePackId"'), PosixPath('"fa871dcc52f689d5c3b8a42d026c7d06.zh-cn","_translationsConfigFile"'), PosixPath('"/home/mymusise/.vscode-server/data/clp/fa871dcc52f689d5c3b8a42d026c7d06.zh-cn/tcf.json","_cacheRoot"'), PosixPath('"/home/mymusise/.vscode-server/data/clp/fa871dcc52f689d5c3b8a42d026c7d06.zh-cn","_resolvedLanguagePackCoreLocation"'), PosixPath('true}'), PosixPath('"/hom

In [2]:
model_name = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1, torch_dtype=torch.bfloat16)

Some weights of the model checkpoint at gpt2 were not used when initializing GPT2ForSequenceClassification: ['h.11.attn.bias', 'h.5.attn.bias', 'h.0.attn.bias', 'h.4.attn.bias', 'h.3.attn.bias', 'h.7.attn.bias', 'h.2.attn.bias', 'h.1.attn.bias', 'h.6.attn.bias', 'h.10.attn.bias', 'h.8.attn.bias', 'h.9.attn.bias']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-

In [3]:
dataset = load_dataset("lvwerra/stack-exchange-paired", data_dir="data/reward", split="train")
print(dataset)

train_dataset = dataset.select(range(10000))

eval_dataset = dataset.select(range(10000, 10100))

Found cached dataset parquet (/home/mymusise/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-ea956f7e49277b88/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


Dataset({
    features: ['qid', 'question', 'date', 'metadata', 'response_j', 'response_k'],
    num_rows: 7441998
})


In [4]:
# Need to do this for gpt2, because it doesn't have an official pad token.
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id
model.config.use_cache = True
num_proc = 24  # Can adjust to be higher if you have more processors.
original_columns = train_dataset.column_names


# Turn the dataset into pairs of post + summaries, where text_j is the preferred question + answer and text_k is the other.
# Then tokenize the dataset.
def preprocess_function(examples):
    new_examples = {
        "input_ids_j": [],
        "attention_mask_j": [],
        "input_ids_k": [],
        "attention_mask_k": [],
    }
    for question, response_j, response_k in zip(examples["question"], examples["response_j"], examples["response_k"]):
        tokenized_j = tokenizer("Question: " + question + "\n\nAnswer: " + response_j, truncation=True)
        tokenized_k = tokenizer("Question: " + question + "\n\nAnswer: " + response_k, truncation=True)

        new_examples["input_ids_j"].append(tokenized_j["input_ids"])
        new_examples["attention_mask_j"].append(tokenized_j["attention_mask"])
        new_examples["input_ids_k"].append(tokenized_k["input_ids"])
        new_examples["attention_mask_k"].append(tokenized_k["attention_mask"])

    return new_examples


# preprocess the dataset and filter out QAs that are longer than 512

max_length = 512

train_dataset = train_dataset.map(
    preprocess_function, batched=True, num_proc=num_proc, remove_columns=original_columns
)
train_dataset = train_dataset.filter(lambda x: len(x["input_ids_j"]) <= max_length and len(x["input_ids_k"]) <= max_length)

eval_dataset = eval_dataset.map(preprocess_function, batched=True, num_proc=num_proc, remove_columns=original_columns)
eval_dataset = eval_dataset.filter(lambda x: len(x["input_ids_j"]) <= max_length and len(x["input_ids_k"]) <= max_length)

Loading cached processed dataset at /home/mymusise/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-ea956f7e49277b88/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-aeceb66b46b56399_*_of_00024.arrow
Loading cached processed dataset at /home/mymusise/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-ea956f7e49277b88/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-dac6338e4d86a38d.arrow


Map (num_proc=24):   0%|          | 0/100 [00:00<?, ? examples/s]

Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

In [5]:
training_args = TrainingArguments(
    output_dir="outputs",
    learning_rate=10e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=1,
    num_train_epochs=1,
    weight_decay=0.001,
    # evaluation_strategy="steps",
    # eval_steps=200,
    save_strategy="steps",
    save_steps=200,
    remove_unused_columns=False,
    bf16=True,
    logging_strategy="steps",
    logging_steps=100,
    optim="adamw_hf",
    lr_scheduler_type="linear",
)

In [6]:

# We need to define a special data collator that batches the data in our j vs k format.
@dataclass
class RewardDataCollatorWithPadding:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    return_tensors: str = "pt"

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, Any]:
        features_j = []
        features_k = []
        for feature in features:
            features_j.append(
                {
                    "input_ids": feature["input_ids_j"],
                    "attention_mask": feature["attention_mask_j"],
                }
            )
            features_k.append(
                {
                    "input_ids": feature["input_ids_k"],
                    "attention_mask": feature["attention_mask_k"],
                }
            )
        batch_j = self.tokenizer.pad(
            features_j,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors=self.return_tensors,
        )
        batch_k = self.tokenizer.pad(
            features_k,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors=self.return_tensors,
        )
        batch = {
            "input_ids_j": batch_j["input_ids"],
            "attention_mask_j": batch_j["attention_mask"],
            "input_ids_k": batch_k["input_ids"],
            "attention_mask_k": batch_k["attention_mask"],
            "return_loss": True,
        }
        return batch


# Define the metric that we'll use for validation.
accuracy = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    predictions, _ = eval_pred
    # Here, predictions is rewards_j and rewards_k.
    # We want to see how much of the time rewards_j > rewards_k.
    predictions = np.argmax(predictions, axis=0)
    labels = np.zeros(predictions.shape)
    return accuracy.compute(predictions=predictions, references=labels)


class RewardTrainer(Trainer):
    # Define how to compute the reward loss. We use the InstructGPT pairwise logloss: https://arxiv.org/abs/2203.02155
    def compute_loss(self, model, inputs, return_outputs=False):
        rewards_j = model(input_ids=inputs["input_ids_j"], attention_mask=inputs["attention_mask_j"])[0]
        rewards_k = model(input_ids=inputs["input_ids_k"], attention_mask=inputs["attention_mask_k"])[0]
        loss = -nn.functional.logsigmoid(rewards_j - rewards_k).mean()
        if return_outputs:
            return loss, {"rewards_j": rewards_j, "rewards_k": rewards_k}
        return loss


# Train the model, woohoo.
trainer = RewardTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    compute_metrics=compute_metrics,
    data_collator=RewardDataCollatorWithPadding(tokenizer=tokenizer, max_length=max_length),
)

trainer.train()

print("Saving last checkpoint of the model")
model.save_pretrained("outputs")

/home/mymusise/pro/stable-diffusion-webui/venv/lib/python3.8/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mymusise. Use `wandb login --relogin` to force relogin


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/mymusise/pro/stable-diffusion-webui/venv/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2382: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
100,0.734600
200,0.707900
300,0.703900
400,0.690100
500,0.695800
600,0.683200
700,0.681500
800,0.661000
900,0.678100
1000,0.663400


/home/mymusise/pro/stable-diffusion-webui/venv/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2382: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/home/mymusise/pro/stable-diffusion-webui/venv/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2382: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/home/mymusise/pro/stable-diffusion-webui/venv/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2382: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/home/mymusise/pro/stable-diffusion-webui/venv/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2382: UserWarning: `max_length` i

Saving last checkpoint of the model


In [16]:
from tqdm import tqdm
import torch


prediction = []
for item in eval_dataset:
    score_j = model(torch.LongTensor([item['input_ids_j']]).cuda())['logits'].tolist()[0][0]
    score_k = model(torch.LongTensor([item['input_ids_k']]).cuda())['logits'].tolist()[0][0]
    prediction.append({"score_j": score_j, "score_k": score_k})

In [24]:
import pandas as pd

df = pd.DataFrame(prediction)
df.head(40)

61


,score_j,score_k
0,2.125000,1.640625
1,1.992188,2.406250
2,1.664062,0.609375
3,0.945312,0.255859
4,1.039062,0.414062
5,0.460938,1.054688
6,0.349609,0.291016
7,0.412109,-0.163086
8,0.279297,0.617188
9,0.255859,0.037598


In [23]:
print(len(df[(df['score_j'] - df['score_k']) > 0]))
print(len(df[(df['score_j'] < df['score_k']) > 0]))

36
25
